In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/mnistdataset/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5.zip"

Archive:  /content/drive/MyDrive/mnistdataset/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5.zip
  inflating: vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5  


In [3]:
from tensorflow.keras.applications.vgg16 import VGG16


In [4]:
base_model = VGG16(include_top=False, weights='/content/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', input_shape=(128, 128, 3), pooling='avg')

In [5]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [6]:
for layer in base_model.layers:
  layer.trainable = False

In [7]:
!unzip "/content/drive/MyDrive/mnistdataset/Copy of cifar-10-img.zip"

Streaming output truncated to the last 5000 lines.
 extracting: cifar-10-img/train/ship/44214.png  
 extracting: cifar-10-img/train/ship/443.png  
 extracting: cifar-10-img/train/ship/44305.png  
 extracting: cifar-10-img/train/ship/44311.png  
 extracting: cifar-10-img/train/ship/4433.png  
 extracting: cifar-10-img/train/ship/44332.png  
 extracting: cifar-10-img/train/ship/44334.png  
 extracting: cifar-10-img/train/ship/44358.png  
 extracting: cifar-10-img/train/ship/44363.png  
 extracting: cifar-10-img/train/ship/44374.png  
 extracting: cifar-10-img/train/ship/44376.png  
 extracting: cifar-10-img/train/ship/44385.png  
 extracting: cifar-10-img/train/ship/44415.png  
 extracting: cifar-10-img/train/ship/44421.png  
 extracting: cifar-10-img/train/ship/44445.png  
 extracting: cifar-10-img/train/ship/44459.png  
 extracting: cifar-10-img/train/ship/4447.png  
 extracting: cifar-10-img/train/ship/44478.png  
 extracting: cifar-10-img/train/ship/44502.png  
 extracting: cifar-10-

In [8]:
train = "/content/cifar-10-img/train"
test = "/content/cifar-10-img/test"

In [10]:
import os
os.listdir(train)

['cat',
 'deer',
 'truck',
 'dog',
 'automobile',
 'horse',
 'ship',
 'airplane',
 'frog',
 'bird']

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_gen = ImageDataGenerator(rescale=1/255.0)

In [12]:
train_generator = data_gen.flow_from_directory(
  train,
  class_mode='categorical',
  shuffle=True,
  batch_size=50000,
  target_size=(32, 32)
)

Found 40079 images belonging to 10 classes.


In [13]:
xtrain, ytrain = train_generator[0]
xtrain.shape

(40079, 32, 32, 3)

In [14]:
ytrain.shape

(40079, 10)

In [15]:
test_generator = data_gen.flow_from_directory(
    test,
    class_mode='categorical',
    shuffle=True,
    batch_size=20000,
    target_size=(32, 32)
)

Found 9921 images belonging to 10 classes.


In [16]:
xtest, ytest = test_generator[0]
xtest.shape

(9921, 32, 32, 3)

In [18]:
from tensorflow.keras.layers import Dense, Flatten, UpSampling2D, MaxPooling2D, Input
from tensorflow.keras.models import Sequential

In [19]:
input_layer = Input(shape=(32, 32, 3))
model = Sequential([
    input_layer,
    UpSampling2D((2, 2)),
    UpSampling2D((2, 2)),
    base_model,
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 up_sampling2d (UpSampling2  (None, 64, 64, 3)         0         
 D)                                                              
                                                                 
 up_sampling2d_1 (UpSamplin  (None, 128, 128, 3)       0         
 g2D)                                                            
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 dense_2 (Dense)             (None, 10)                3

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
h = model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=10, batch_size=512)

Epoch 1/10
79/79 [==============================] - 92s 916ms/step - loss: 1.9346 - accuracy: 0.3428 - val_loss: 1.5784 - val_accuracy: 0.4743
Epoch 2/10
79/79 [==============================] - 70s 891ms/step - loss: 1.4338 - accuracy: 0.5130 - val_loss: 1.3199 - val_accuracy: 0.5479
Epoch 3/10
79/79 [==============================] - 72s 909ms/step - loss: 1.2555 - accuracy: 0.5683 - val_loss: 1.2014 - val_accuracy: 0.5812
Epoch 4/10
79/79 [==============================] - 72s 910ms/step - loss: 1.1649 - accuracy: 0.5963 - val_loss: 1.1400 - val_accuracy: 0.6089
Epoch 5/10
79/79 [==============================] - 72s 914ms/step - loss: 1.1079 - accuracy: 0.6186 - val_loss: 1.0945 - val_accuracy: 0.6251
Epoch 6/10
79/79 [==============================] - 73s 921ms/step - loss: 1.0668 - accuracy: 0.6320 - val_loss: 1.0734 - val_accuracy: 0.6274
Epoch 7/10
79/79 [==============================] - 73s 923ms/step - loss: 1.0388 - accuracy: 0.6419 - val_loss: 1.0454 - val_accuracy: 0.6361

In [21]:
base_model2 = VGG16(include_top=False, weights='/content/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', input_shape=(128, 128, 3))
for layers in base_model2.layers:
  layers.trainable=False

In [27]:
from tensorflow.keras.layers import Dropout
for layers in base_model2.layers[len(base_model2.layers)-5:]:
  layers.trainable=True
inp_layer = Input(shape=(32, 32, 3))
newmodel = Sequential([
    inp_layer,
    UpSampling2D((2, 2)),
    UpSampling2D((2, 2)),
    base_model2,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='softmax'),
])
newmodel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 up_sampling2d_8 (UpSamplin  (None, 64, 64, 3)         0         
 g2D)                                                            
                                                                 
 up_sampling2d_9 (UpSamplin  (None, 128, 128, 3)       0         
 g2D)                                                            
                                                                 
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense_10 (Dense)            (None, 256)               2097408   
                                                                 
 dropout_4 (Dropout)         (None, 256)              

In [28]:
newmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
h1 = newmodel.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=10, batch_size=512)

Epoch 1/10
79/79 [==============================] - 81s 934ms/step - loss: 2.0958 - accuracy: 0.1906 - val_loss: 1.6571 - val_accuracy: 0.3686
Epoch 2/10
79/79 [==============================] - 75s 953ms/step - loss: 1.2794 - accuracy: 0.5352 - val_loss: 1.0689 - val_accuracy: 0.6231
Epoch 3/10
79/79 [==============================] - 77s 977ms/step - loss: 0.8646 - accuracy: 0.6973 - val_loss: 0.8391 - val_accuracy: 0.7041
Epoch 4/10
79/79 [==============================] - 69s 878ms/step - loss: 0.6751 - accuracy: 0.7700 - val_loss: 0.7119 - val_accuracy: 0.7503
Epoch 5/10
79/79 [==============================] - 78s 988ms/step - loss: 0.5211 - accuracy: 0.8234 - val_loss: 0.6174 - val_accuracy: 0.7877
Epoch 6/10
79/79 [==============================] - 78s 991ms/step - loss: 0.4343 - accuracy: 0.8513 - val_loss: 0.5989 - val_accuracy: 0.8024
Epoch 7/10
79/79 [==============================] - 70s 894ms/step - loss: 0.3341 - accuracy: 0.8873 - val_loss: 0.6656 - val_accuracy: 0.7921